## Семинар 12. Линейная регрессия (часть 3).

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
sns.set()

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

### Задача №1

В файле "House_prices_corrected.csv" представлены характеристики различных домов (стоимость, площадь, количество комнат, год постройки и тп, описание признаков можно найти по ссылке [__Ames Housing dataset__](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)).

Изучить линейную зависимость стоимости домов (SalePrice) от всех остальных показателей.

In [3]:
data = pd.read_csv('House_prices_corrected.csv')
#House_prices_corrected: заполнены пропуски в данных и устранена мультиколлинеарность (см. предыдущий семинар)

In [4]:
data.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,222264
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,217555
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,270819
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,176732
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,310680


In [24]:
x = data.drop(['SalePrice'], 1)
y = data['SalePrice']
ln_y = np.log(y) #почему берем ln - чтобы остатки регрессии были нормальными

### Деление выборки на обучающую и тестовую

Портной научился хорошо шить костюмы для Смита.
Пока он шьет для Смита – все хорошо.
Но если он будет шить для Джонса по тем же параметрам, что и для Смита, результат может быть намного хуже.

Итак, разделим выборку на обучающую и тестовую. Разделим случайным образом 75% на 25%:

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=10)

У моделей из `sklearn` есть методы `fit` и `predict`. 

`fit` принимает на вход обучающую выборку и вектор целевых переменных и обучает модель, `predict`, будучи вызванным после обучения модели, возвращает предсказание на выборке.

**R2 - чем больше, тем лучше**

In [7]:
from sklearn.metrics import mean_squared_error, r2_score

lr = LinearRegression() #по умолчанию в модели регрессии есть константа
lr.fit(x_train, y_train)

y_hat_test = lr.predict(x_test)
print('Using Y:')
print('Test MSE %.3f' % mean_squared_error(y_test, y_hat_test))
print('Test R2 %.3f' % r2_score(y_test, y_hat_test))

Using Y:
Test MSE 1304270005.734
Test R2 0.771


Для сравнения:

In [8]:
y_hat_train = lr.predict(x_train)
print("Train MSE = %.3f" % mean_squared_error(y_train, y_hat_train))
print("Train R2 = %.3f" % r2_score(y_train, y_hat_train)) 

Train MSE = 1342194893.254
Train R2 = 0.813


Коэффициенты регрессии, посчитанные по трейну при 32 факторах без константы

In [9]:
lr.coef_

array([-8.87448148e+01,  1.36196368e+02,  4.22706526e-01,  1.47452840e+04,
        6.33991760e+03,  3.72818710e+02,  1.30512834e+02,  1.68235714e+01,
        5.29744899e+01,  2.83012971e+01,  2.69175677e+01,  1.69867583e+01,
        2.54490673e+01, -2.75573313e+00,  3.96800924e+01, -1.57694312e+03,
        7.03211520e+02,  4.88879652e+03, -2.62146188e+03, -6.79147570e+03,
       -3.39564579e+03,  4.73489470e+03,  3.71926057e+01,  2.49076538e+00,
       -8.75749741e+00, -1.80111006e+01,  2.30343428e+01,  9.11847013e+00,
        2.22007223e+01, -7.41983127e-02, -1.83515418e+01, -1.01896943e+03])

Константа лежит тут

In [10]:
lr.intercept_

966374.9095583935

##### Если будем использовать логарифмирование данных, получим

**Важно** не забыть экспоненту во второй строчке!

In [11]:
lr.fit(x_train, np.log(y_train))
y_hat_test = np.exp(lr.predict(x_test))
print('Using logY:')
print('Test MSE %.3f' % mean_squared_error(y_test, y_hat_test))
print('Test R2 %.3f' % r2_score(y_test, y_hat_test))

Using logY:
Test MSE 866920715.293
Test R2 0.848


Итак, мы обучили модель и посчитали ее качество на тестовой выборке.

### Кросс-валидация

Принцип кросс-валидации изображен на рисунке

<img src="https://docs.splunk.com/images/thumb/e/ee/Kfold_cv_diagram.png/1200px-Kfold_cv_diagram.png" width=50%>

Так как **MSE - "чем меньше, тем лучше"**, а **для кросс-валидации нужна метрика "чем больше, тем лучше"**, то будем использовать neg_mean_squared_error

In [12]:
cv_scores = cross_val_score(lr, x, y, cv=10, scoring="neg_mean_squared_error")
cv_scores

array([-9.10573226e+08, -1.02586915e+09, -1.55473556e+09, -1.02731631e+09,
       -2.02443946e+09, -1.58980221e+09, -1.19599953e+09, -1.11111813e+09,
       -3.31345319e+09, -1.19120293e+09])

Обратите внимание на то, что результаты получились отрицательными. Это соглашение в `sklearn` (скоринговую функцию нужно максимизировать). Поэтому все стандартные скореры называются `neg_*`, например, `neg_mean_squared_error`.

In [13]:
print("Mean CV MSE = %.4f" % np.mean(-cv_scores))

Mean CV MSE = 1494450970.6308


Мы всегда можем определить свою метрику и использовать ее, например, в `cross_val_score`. Для этого нужно воспользоваться `sklearn.metrics.make_scorer`.

Рассмотрим, например, $R^2$.

In [14]:
from sklearn.metrics import make_scorer

def r2_squared(y_true, y_pred):
    r2_coef = r2_score(y_true, y_pred)
    return r2_coef

r2_scorer = make_scorer(r2_squared, greater_is_better=True) #greater_is_better влияет на знаки метрик в cv_scores

In [15]:
cv_scores = cross_val_score(lr, x, y, cv=10, scoring=r2_scorer) #scoring="r2"
print("Mean CV R2 = %.4f" % np.mean(cv_scores))

Mean CV R2 = 0.7825


In [16]:
cv_scores = cross_val_score(lr, x, ln_y, cv=10, scoring=r2_scorer) #scoring="r2"
print("Mean CV R2 = %.4f" % np.mean(cv_scores))

Mean CV R2 = 0.8504


### Отбор признаков с помощью кросс-валидации (greedy algorithm)

Идея не отличается от реализации с прошлого семинара. Добавили кросс-валидацию. Обратим внимание, что тут метрика по умолчанию neg_mean_squared. 5 фолдов.

In [17]:
# Возвращает усредненную MSE метрику (чем меньше, тем лучше)
def calc_kfold_validation(x, y):
    lr = LinearRegression()
    cv_scores = cross_val_score(lr, x, y, cv=5, scoring="neg_mean_squared_error")
    return np.mean(-cv_scores)

In [26]:
def select_best_combination(x, y):
    current_factors = x.columns.to_list() #сначала создаем список всех наименований столбцов
    metric_base = calc_kfold_validation(x[current_factors], y)

    while 1 == 1:
        res = pd.Series(index=current_factors) #создаем Series c индексом current_factors и значениями = Nan
        for factor in current_factors:
            res.loc[factor] = calc_kfold_validation(x[list(set(current_factors)-{factor})], y)
                #вместо Nan в res записываем метрику, соответствующую модели без данного столбца
        # так как используем MSE 
        res = res.sort_values(ascending=True) #сортируем res по возрастанию 
        if res.iloc[0] < metric_base:
            current_factors.remove(res.index.values[0])
            metric_base = res.iloc[0]
        else:
            break
            
    return current_factors, calc_kfold_validation(x[current_factors], y)

In [19]:
ln_y_train, ln_y_test = np.log(y_train), np.log(y_test)

In [20]:
current_factors, _ = select_best_combination(x_train, ln_y_train) #делаем отбор признаков на train (по ln_y_train)
current_factors

/home/andrew/miniconda3/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/home/andrew/miniconda3/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/home/andrew/miniconda3/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/home/andrew/miniconda3/envs/da/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence t

['MSSubClass',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'Fireplaces',
 'GarageArea',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'YrSold']

In [21]:
set(x.columns.to_list()) - set(current_factors)

{'EnclosedPorch',
 'KitchenAbvGr',
 'LotFrontage',
 'MiscVal',
 'MoSold',
 'OpenPorchSF',
 'WoodDeckSF'}

#### Сравним модель до и после отбора признаков

In [22]:
k = x_train.shape[1] + 1 #количество факторов (с учетом const)
n = x_test.shape[0]
lr = LinearRegression()
lr.fit(x_train, ln_y_train)

ln_y_hat_test = lr.predict(x_test)
print('Test R2 %.3f' % r2_score(ln_y_test, ln_y_hat_test))
print('Test AIC %.3f' % (2*k + n * np.log(np.sum((ln_y_test - ln_y_hat_test) ** 2))) )

Test R2 0.858
Test AIC 835.847


In [23]:
k = x_train[current_factors].shape[1] + 1 #количество факторов (с учетом const)
n = x_test.shape[0]
lr = LinearRegression()
lr.fit(x_train[current_factors], ln_y_train)

ln_y_hat_test = lr.predict(x_test[current_factors])
print('After greedy algorithm')
print('Test R2 %.3f' % r2_score(ln_y_test, ln_y_hat_test))
print('Test AIC %.3f' % (2*k + n * np.log(np.sum((ln_y_test - ln_y_hat_test) ** 2))) )

After greedy algorithm
Test R2 0.856
Test AIC 825.163


Чем больше признаков, тем больше R2